## Simulate using SEIRD epidemiological model

The first step to begin the simulation is to determine the differential equations reggarding the model. The SEIRD model divides the population into 5 groups, those suseptible to the disease $S(t)$, those who got the disease, but didn't develop symptoms yet $E(t)$, the ones who are infected and already show symptoms $I(t)$, those who recovered from the disease $R(t)$ and acquried immunicity and at last, the ones who sadly died from the disease $D(t)$.

The behavior of these populations is governed by exchanging rates, that is, rates in which individuals leave a group and get to another group.

![models.png](models.png)

With that in hands, we build the equations for the $S$ population

$$\frac{dS}{dt} = -\beta  \frac{S(t)I(t)}{N}$$

the $E$ population

$$\frac{dE}{dt} = \beta  \frac{S(t)I(t)}{N} - \alpha E(t)$$

the $I$ population

$$\frac{dI}{dt} = \alpha E(t) - \gamma I(t) - \delta I(t)$$

the $R$ population

$$\frac{dR}{dt} = \gamma I(t)$$

and the $D$ population

$$\frac{dD}{dt} = \delta I(t)$$

In [139]:
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd
from matplotlib import pyplot as plt

In [119]:
# Population of india
N = 1380004385

In [120]:
#data feeding
df = pd.read_csv("india.csv", header = 0) # dataset
df.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths
0,2020-03-11,India,NaN,21,78,62,4,1
1,2020-03-12,India,NaN,21,78,73,4,1
2,2020-03-13,India,NaN,21,78,82,4,2
3,2020-03-14,India,NaN,21,78,102,4,2
4,2020-03-15,India,NaN,21,78,113,13,2


In [121]:
available = len(df.index)
print("Available", available, "days")

Available 89 days


In [122]:
start_date = df['Date'].iloc[0]
confirmed = df['Confirmed'].values.tolist()
recovered = df['Recovered'].values.tolist()
deaths = df['Deaths'].values.tolist()
infected = [ c - r - d for (c, r, d) in zip(confirmed, recovered, deaths)]

In [106]:
print(confirmed)

[62, 73, 82, 102, 113, 119, 142, 156, 194, 244, 330, 396, 499, 536, 657, 727, 887, 987, 1024, 1251, 1397, 1998, 2543, 2567, 3082, 3588, 4778, 5311, 5916, 6725, 7598, 8446, 9205, 10453, 11487, 12322, 13430, 14352, 15722, 17615, 18539, 20080, 21370, 23077, 24530, 26283, 27890, 29451, 31324, 33062, 34863, 37257, 39699, 42505, 46437, 49400, 52987, 56351, 59695, 62808, 67161, 70768, 74292, 78055, 81997, 85784, 90648, 95698, 100328, 106475, 112028, 118226, 124794, 131423, 138536, 144950, 150793, 158086, 165386, 173491, 181827, 190609, 198370, 207191, 216824, 226713, 236184, 246622, 257486]


In [107]:
print(start_date)

2020-03-11


In [123]:
#Helper Function

t_max = available + 7

date = np.array('2020-03-11', dtype=np.datetime64) 
dates = date + np.arange(t_max)
str_dates = []
for i in dates:
    str_dates.append(str(i))

print(str_dates)
print(len(str_dates))

['2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20', '2020

In [124]:
def seird_model(init_vals, params, t):

    DATE_0, S_0, E_0, I_0, R_0, D_0 = init_vals
    DATE, S, E, I, R, D = [DATE_0], [S_0], [E_0], [I_0], [R_0], [D_0]
    alpha, beta, gamma, delta = params
    
    ##### Create next t days ######
    date = np.array(DATE, dtype=np.datetime64)
    dates = date + np.arange(len(t))
    str_dates = []
    for i in dates:
        str_dates.append(str(i))
        
    ##### End creating t days #####
    for tic in t[1:]:

        DATE.append(str_dates[tic])

        next_S = S[-1] - ( beta * (S[-1] / N) * I[-1] )  # Susceptible
        next_E = E[-1] + ( beta * (S[-1] / N) * I[-1] ) - ( alpha * E[-1] ) # Exposed
        next_I = I[-1] + ( alpha * E[-1] ) - ( gamma * I[-1] ) - ( delta * I[-1] ) # Infected
        next_R = R[-1] + ( gamma * I[-1] ) # Recovered
        next_D = D[-1] + ( delta * I[-1] ) # Deaths
        
        S.append(next_S)
        E.append(next_E)
        I.append(next_I)
        R.append(next_R)
        D.append(next_D)


    return np.stack([DATE, S, E, I, R, D]).T

In [125]:
# Spliting the data for one day rolling window approach

train_min = 1
train_max = available + 1

inf_train = []
rec_train = []
death_train = []

for i in range(train_min+1, train_max):
    j = i - 2
    inf_train.append(infected[j:i])
    rec_train.append(recovered[j:i])
    death_train.append(deaths[j:i])

In [127]:
for i in range(len(rec_train)): 
    print(i, rec_train[i])

0 [4, 4]
1 [4, 4]
2 [4, 4]
3 [4, 13]
4 [13, 13]
5 [13, 14]
6 [14, 14]
7 [14, 15]
8 [15, 20]
9 [20, 23]
10 [23, 27]
11 [27, 27]
12 [27, 40]
13 [40, 43]
14 [43, 45]
15 [45, 73]
16 [73, 84]
17 [84, 95]
18 [95, 102]
19 [102, 123]
20 [123, 148]
21 [148, 191]
22 [191, 192]
23 [192, 229]
24 [229, 229]
25 [229, 375]
26 [375, 421]
27 [421, 506]
28 [506, 620]
29 [620, 774]
30 [774, 969]
31 [969, 1080]
32 [1080, 1181]
33 [1181, 1359]
34 [1359, 1432]
35 [1432, 1768]
36 [1768, 2041]
37 [2041, 2463]
38 [2463, 2854]
39 [2854, 3273]
40 [3273, 3975]
41 [3975, 4370]
42 [4370, 5012]
43 [5012, 5498]
44 [5498, 5939]
45 [5939, 6523]
46 [6523, 7137]
47 [7137, 7747]
48 [7747, 8437]
49 [8437, 9068]
50 [9068, 10007]
51 [10007, 10819]
52 [10819, 11775]
53 [11775, 12847]
54 [12847, 14142]
55 [14142, 15331]
56 [15331, 16776]
57 [16776, 17887]
58 [17887, 19301]
59 [19301, 20969]
60 [20969, 22549]
61 [22549, 24420]
62 [24420, 26400]
63 [26400, 27969]
64 [27969, 30258]
65 [30258, 34224]
66 [34224, 36795]
67 [36795, 3

In [128]:
t2 = np.arange(0, 3, 1)
print(t2)

[0 1 2]


In [129]:
last5_vals = []
last5_params = []

t_incub = 5  # Assumption 5 days

for sample in range(len(inf_train)):

    i_0 = inf_train[sample][0]
    i_1 = inf_train[sample][1]
    i_diff = i_1 - i_0


    r_0 = rec_train[sample][0]
    r_1 = rec_train[sample][1]
    r_diff = r_1 - r_0


    d_0 = death_train[sample][0]
    d_1 = death_train[sample][1]
    d_diff = d_1 - d_0


    if (sample == 0):

        e_0 = confirmed[t_incub] - confirmed[sample]
        e_1 = confirmed[t_incub + 1] - confirmed[sample + 1]
        e_diff = e_1 - e_0
        print(e_0,e_1)
        s_0 = N - e_0 - i_0 - r_0 - d_0


    alpha = ( i_diff + r_diff + d_diff) / e_0  # alpha
    beta = ( ( ( e_diff + i_diff + r_diff + d_diff) * N ) / ( s_0 * i_0 ) ) # beta
    gamma = r_diff / i_0  # gamma
    delta = d_diff / i_0  # delta

    init_vals = str_dates[sample], s_0, e_0, i_0, r_0, d_0
    params = alpha, beta, gamma, delta
    pred = seird_model(init_vals, params, t2)
    # print(pred)
    # Date: 0, S:1, E:2, I:3, R:4, D:5
    s_0 = float(pred[1][1])
    
    e_0 = float(pred[1][2])
    e_1 = float(pred[2][2])
    e_diff = e_1 - e_0

    # Store Last five cases for predecting future
    if ((len( inf_train )  - sample ) <= 5):
        print(str_dates[sample])
        last5_vals.append(init_vals)
        last5_params.append(params)

57 69
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06


In [130]:
for v in last5_vals: 
    print(v)

('2020-06-02', 1352226363.8535762, 566445.1464237358, 101077, 100285, 5829)
('2020-06-03', 1352193074.0114532, 590101.9885468835, 106665, 104071, 6088)
('2020-06-04', 1352158090.928749, 615196.071251014, 111900, 108450, 6363)
('2020-06-05', 1352122230.3942435, 641585.6057565685, 116302, 113233, 6649)
('2020-06-06', 1352084399.4108748, 668978.5891252063, 120981, 118695, 6946)


In [131]:
for p in last5_params: 
    print(p)

(0.017006059740856044, 0.3295397362154528, 0.03745659249878805, 0.0025624029205457226)
(0.016758120107935072, 0.3281672430711372, 0.04105376646510102, 0.0025781652838325598)
(0.015395091813152389, 0.32066901469849524, 0.04274352100089365, 0.002555853440571939)
(0.01626906823710818, 0.3254934778038102, 0.04696393871128614, 0.0025536964110677374)
(0.016239682669375668, 0.325330876835034, 0.04259346508955952, 0.002157363552954596)


Based on each learned parameter, we will predict values from 10 April to 16 April

In [134]:
# How many days should we use ?? and Why ?? 
t7 = np.arange(0, 13, 1)
print(t7)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [137]:
for z in range(1, len(last5_params)+1):
    results = seird_model(last5_vals[z-1], last5_params[z-1], t7)
    t_z = np.arange(7 - z, 14 - z, 1)
    print("Predictions using", last5_vals[z-1][0], "parameters\n")
    print("-----------------------------------------------------")
    print("Date", "\t\tInfected", "\tRecovered", "\tDeaths")
    print("-----------------------------------------------------")
    for day in t_z:
        print(results[day][0], "\t", int(float(results[day][3])), "\t\t", int(float(results[day][4])), "\t\t", int(float(results[day][5])) )
    print("----------------------------------------------------\n\n")

Predictions using 2020-06-02 parameters

-----------------------------------------------------
Date 		Infected 	Recovered 	Deaths
-----------------------------------------------------
2020-06-08 	 137629 		 126284 		 7607
2020-06-09 	 144548 		 131439 		 7960
2020-06-10 	 151749 		 136853 		 8330
2020-06-11 	 159251 		 142537 		 8719
2020-06-12 	 167071 		 148502 		 9127
2020-06-13 	 175229 		 154760 		 9555
2020-06-14 	 183743 		 161324 		 10004
----------------------------------------------------


Predictions using 2020-06-03 parameters

-----------------------------------------------------
Date 		Infected 	Recovered 	Deaths
-----------------------------------------------------
2020-06-08 	 134895 		 128196 		 7603
2020-06-09 	 141225 		 133734 		 7950
2020-06-10 	 147815 		 139532 		 8314
2020-06-11 	 154681 		 145600 		 8696
2020-06-12 	 161836 		 151951 		 9094
2020-06-13 	 169295 		 158595 		 9512
2020-06-14 	 177076 		 165545 		 9948
--------------------------------------------